In [1]:
import numpy as np
import pandas as pd
import plotly_express as px
import matplotlib.pyplot as plt
from Invoice import Invoice
from Enterprise import Enterprise

In [2]:
enterprise_info=pd.read_csv('./C/2_info.csv')
N_enterprise=enterprise_info.shape[0]

In [3]:
enterprise_dic={}
for i in range(N_enterprise):
    number = enterprise_info['企业代号'][i]
    name = enterprise_info['企业名称'][i]
    enterprise_object=Enterprise(number,name,credit_rating=None,break_contract=None)
    enterprise_dic[number]=enterprise_object

In [9]:
invoice_in = pd.read_csv('./C/2_in.csv')
N_in = invoice_in.shape[0]

In [10]:
from datetime import date
def get_date(date_str):
    y,m,d=date_str.split('/')
    if len(m) != 2:
        m = '0' + m
    if len(d) != 2:
        d = '0' + d
    return date.fromisoformat(y+'-'+m+'-'+d)

In [11]:
for i in range(N_in):
    enterprise_name = invoice_in['企业代号'][i]
    number = invoice_in['发票号码'][i]
    date_str = invoice_in['开票日期'][i]
    date = get_date(date_str)
    self_enterprise = enterprise_dic[enterprise_name]
    partner = invoice_in['销方单位代号'][i]
    amount = invoice_in['金额'][i]
    tax = invoice_in['税额'][i]
    sum_money = invoice_in['价税合计'][i]
    state_available_str = invoice_in['发票状态'][i]
    state_available = True if state_available_str == '有效发票' else False
    invoice_object = Invoice(number,date,self_enterprise,partner,amount,tax,sum_money,state_available,True)
    self_enterprise.add_invoice(invoice_object)

In [12]:
invoice_out = pd.read_csv('./C/2_out.csv')
N_out = invoice_out.shape[0]

In [13]:
for i in range(N_out):
    enterprise_name = invoice_out['企业代号'][i]
    number = invoice_out['发票号码'][i]
    date_str = invoice_out['开票日期'][i]
    date = get_date(date_str)
    self_enterprise = enterprise_dic[enterprise_name]
    partner = invoice_out['购方单位代号'][i]
    amount = invoice_out['金额'][i]
    tax = invoice_out['税额'][i]
    sum_money = invoice_out['价税合计'][i]
    state_available_str = invoice_out['发票状态'][i]
    state_available = True if state_available_str == '有效发票' else False
    invoice_object = Invoice(number,date,self_enterprise,partner,amount,tax,sum_money,state_available,False)
    self_enterprise.add_invoice(invoice_object)

In [14]:
for enterprise in enterprise_dic.values():
    enterprise.invoice_list.sort(key=lambda x:x.date)

In [18]:
frame = pd.DataFrame(columns=['日期','资金','企业代号'])
for enterprise in enterprise_dic.values():
    sum = 0
    current_date = enterprise.invoice_list[0].date
    for invoice in enterprise.invoice_list:
        if invoice.state_avaliable == False:
            continue
        temp = invoice
        if invoice.date!=current_date:
            frame = frame.append([{'日期':current_date,'资金':sum,'企业代号':enterprise.number}],ignore_index=True)
            current_date = invoice.date
        if invoice.buy_in == True:
            sum = sum - invoice.sum_money
        else:
            sum = sum + invoice.amount
    frame = frame.append([{'日期':temp.date,'资金':sum,'企业代号':enterprise.number}],ignore_index=True)
frame.to_csv("./C/2_收益波动_日期.csv",index=False,sep=',',encoding='utf_8_sig')

In [21]:
graph = px.line(frame, x="日期", y="资金",color='企业代号',  render_mode="auto")
graph.write_html('./2_graph/total.html')

In [22]:
frame1 = pd.DataFrame(columns=['日期','资金','企业代号','信誉评级'])
for enterprise in enterprise_dic.values():
    temp = frame[frame.企业代号 == enterprise.number].reset_index(drop=True)
    first_date = temp['日期'][0]
    for i in range(temp.shape[0]):
        frame1 = frame1.append([{'日期':(temp['日期'][i]-first_date).days,'资金':temp['资金'][i],'企业代号':temp['企业代号'][i]}],ignore_index=True)

In [23]:
frame1.to_csv("./C/2_收益波动.csv",index=False,sep=',',encoding='utf_8_sig')
#frame1 = pd.read_csv("./C/2_收益波动.csv")